# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("..\output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,276.09,72,13,5.32,GS,1707083260
1,1,novodvinsk,64.4165,40.8122,264.01,93,100,4.25,RU,1707083260
2,2,stanley,54.8680,-1.6985,285.11,77,75,12.86,GB,1707083261
3,3,olonkinbyen,70.9221,-8.7187,267.38,77,100,11.88,SJ,1707083261
4,4,ushuaia,-54.8000,-68.3000,288.96,67,75,2.06,AR,1707082963


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.scatter(x='Lng', y='Lat', c='Humidity', hover_cols=['City'], 
                                       size='Humidity', alpha=0.5, cmap='viridis', 
                                       title='City Map with Humidity')

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[city_data_df['Cloudiness'] == 0]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
print(ideal_weather_df.head())

    City_ID       City      Lat       Lng  Max Temp  Humidity  Cloudiness  \
8         8  jiayuguan  39.8167   98.3000    265.93        64           0   
12       12       lebu -37.6167  -73.6500    290.65        69           0   
28       28     kodiak  57.7900 -152.4072    274.88        80           0   
31       31   akureyri  65.6835  -18.0878    261.11        79           0   
44       44     bethel  41.3712  -73.4140    279.68        48           0   

    Wind Speed Country        Date  
8         1.20      CN  1707083263  
12        4.78      CL  1707083265  
28        4.12      US  1707083273  
31        1.54      IS  1707083274  
44        5.14      US  1707083279  


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,jiayuguan,CN,39.8167,98.3000,64,
12,lebu,CL,-37.6167,-73.6500,69,
28,kodiak,US,57.7900,-152.4072,80,
31,akureyri,IS,65.6835,-18.0878,79,
44,bethel,US,41.3712,-73.4140,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "categories": "accommodation.hotel",
    "limit": 1,  
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius}@{lng},{lat}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jiayuguan - nearest hotel: No hotel found
lebu - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
akureyri - nearest hotel: No hotel found
bethel - nearest hotel: No hotel found
nova sintra - nearest hotel: No hotel found
hood river - nearest hotel: No hotel found
jinghong - nearest hotel: No hotel found
chibougamau - nearest hotel: No hotel found
cam ranh - nearest hotel: No hotel found
crane - nearest hotel: No hotel found
aoulef - nearest hotel: No hotel found
bam - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
jamestown - nearest hotel: No hotel found
dalbandin - nearest hotel: No hotel found
lazaro cardenas - nearest hotel: No hotel found
fabens - nearest hotel: No hotel found
ghat - nearest hotel: No hotel found
cabo san lucas - nearest hotel: No hotel found
san luis de la loma - nearest hotel: No hotel found
brisas de zicatela - nearest hotel: No hotel found
greenvi

,City,Country,Lat,Lng,Humidity,Hotel Name
8,jiayuguan,CN,39.8167,98.3000,64,No hotel found
12,lebu,CL,-37.6167,-73.6500,69,No hotel found
28,kodiak,US,57.7900,-152.4072,80,No hotel found
31,akureyri,IS,65.6835,-18.0878,79,No hotel found
44,bethel,US,41.3712,-73.4140,48,No hotel found
...,...,...,...,...,...,...
527,acapulco de juarez,MX,16.8634,-99.8901,51,No hotel found
528,aswan,EG,24.0934,32.9070,24,No hotel found
541,goshen,US,41.5823,-85.8344,54,No hotel found
552,es castell,ES,39.8795,4.2854,100,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.scatter(x="Lng", y="Lat", size="Humidity", color="Blue",
                                   hover_cols=["City", "Country", "Hotel Name"],
                                   alpha=0.5, title="City Weather Data with Hotels")

# Display the map
map_plot

:Scatter   [Lng]   (Lat,Humidity,City,Country,Hotel Name)